In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
all_sites = pd.read_csv('../data/all_sites.csv')

In [3]:
all_sites2 = all_sites[3100:6200]

In [4]:
# Select model
nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(nlp_model)
model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

# Classifier
classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer)

In [8]:
def sentiment_analysis_bert_base_multilingual_uncased(review):
    global_stars = []
    n = 1500
    
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]    
    global_stars = (classifier(review_splitted))
    
    return global_stars


In [9]:
def apply_sentiment_model(df):
    try:
        df['stars'] = df['text'].apply(lambda x: sentiment_analysis_bert_base_multilingual_uncased(x))
        
    except RuntimeError:
        print(df['game'])
        df['stars'] = df['text'].apply(lambda x: np.nan)
    
    finally:
        df.to_csv('../data/labeled_texts2.csv', index=False)
    
    return df

In [10]:
apply_sentiment_model(all_sites2)

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,site,url_link,author,game,company,genre,platform,text,score,stars
3100,Vandal,https://vandal.elespanol.com/analisis/android/...,Alberto González,The Dark Knight Rises - Análisis,Gameloft,Acción Beat'em up,Android iPhone,"Batman, el poderoso hombre multimillonario env...",8.9,"[{'label': '4 stars', 'score': 0.5204374790191..."
3101,Gamereactor,https://www.gamereactor.es/dying-light-the-fol...,Jerry Fogselius,Dying Light: The Following,Warner Bros. Interactive Entertainment,Acción,"PC, PS4, Xbox One",Una de las mayores sorpresas del año pasado fu...,9.0,"[{'label': '4 stars', 'score': 0.5572187900543..."
3102,3D Juegos,https://www.3djuegos.com/juegos/analisis/16672...,Sergio Martín,Deus Ex: Human Revolution Director's Cut - Aná...,None,None,PC PS3 X360 Wii U Mac,Deus Ex: Human Revolution es una de las mejore...,9.0,"[{'label': '4 stars', 'score': 0.5699344277381..."
3103,3D Juegos,https://www.3djuegos.com/juegos/analisis/24559...,Jesús Bella,Pokémon Sol / Pokémon Luna - Análisis,None,None,3DS,El eterno debate: ¿un pokémon para expertos o ...,9.0,"[{'label': '2 stars', 'score': 0.3973818719387..."
3104,meristation,https://as.com/meristation/2009/02/03/analisis...,Will van Dijk,Pikmin - New Play Control,Nintendo,Strategy,Wii,Poco a poco se van mitigando las dudas sobre l...,7.0,"[{'label': '3 stars', 'score': 0.5458763837814..."
...,...,...,...,...,...,...,...,...,...,...
6195,revogamers,https://www.revogamers.net/analisis-w/neon-chr...,Arturo Albero,Neon Chrome,None,None,Nintendo Switch,10tons ha traído a Nintendo Switch sus tres ju...,6.0,"[{'label': '4 stars', 'score': 0.4140677750110..."
6196,Gamereactor,https://www.gamereactor.es/the-curious-tale-of...,Mikael Sundberg,The Curious Tale of the Stolen Pets,None,Puzle,None,"Fast Travel Games es un estudio pequeñito, per...",6.0,"[{'label': '4 stars', 'score': 0.4538985490798..."
6197,Vandal,https://vandal.elespanol.com/analisis/pc/my-fr...,Juan Rubio,My Friend Pedro - Análisis,Devolver Digital,Acción Arcade,PC Switch Xbox One PS4,"Cada género tiene sus particularidades y, por ...",8.0,"[{'label': '4 stars', 'score': 0.6285300254821..."
6198,meristation,https://as.com/meristation/2010/03/24/analisis...,Salva Fernàndez,Prison Break: The Conspiracy,Deep Silver,"Acción, Aventura",PC 360 PS3,Las adaptaciones van y vienen. De Libros a vid...,5.0,"[{'label': '4 stars', 'score': 0.4500963985919..."


In [11]:
def stars_mean_to_score(review):
    
    points = []

    for classification in review:

        #Mean of fragment
        grade = int(classification['label'].split(' ')[0])
        points.append(grade)  
    score = round(np.mean(points), 2)

    return(score)   

In [12]:
all_sites2['stars_mean'] = all_sites2['stars'].apply(lambda x: stars_mean_to_score(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
def score_labels(score):
    scorelabel = f"'{round(score / 2)} starts'"
    return scorelabel

In [14]:
all_sites2['score_label'] = all_sites2['score'].apply(lambda x: score_labels(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
def score_half(score):
    return score / 2

In [16]:
all_sites2['score_adj'] = all_sites2['score'].apply(lambda x: score_half(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
all_sites2.to_csv('../data/scored_texts2.csv', index=False)

In [17]:
all_sites2

,site,url_link,author,game,company,genre,platform,text,score,stars,stars_mean,score_label,score_adj
3100,Vandal,https://vandal.elespanol.com/analisis/android/...,Alberto González,The Dark Knight Rises - Análisis,Gameloft,Acción Beat'em up,Android iPhone,"Batman, el poderoso hombre multimillonario env...",8.9,"[{'label': '4 stars', 'score': 0.5204374790191...",4.00,'4 starts',4.45
3101,Gamereactor,https://www.gamereactor.es/dying-light-the-fol...,Jerry Fogselius,Dying Light: The Following,Warner Bros. Interactive Entertainment,Acción,"PC, PS4, Xbox One",Una de las mayores sorpresas del año pasado fu...,9.0,"[{'label': '4 stars', 'score': 0.5572187900543...",3.75,'4 starts',4.50
3102,3D Juegos,https://www.3djuegos.com/juegos/analisis/16672...,Sergio Martín,Deus Ex: Human Revolution Director's Cut - Aná...,None,None,PC PS3 X360 Wii U Mac,Deus Ex: Human Revolution es una de las mejore...,9.0,"[{'label': '4 stars', 'score': 0.5699344277381...",3.88,'4 starts',4.50
3103,3D Juegos,https://www.3djuegos.com/juegos/analisis/24559...,Jesús Bella,Pokémon Sol / Pokémon Luna - Análisis,None,None,3DS,El eterno debate: ¿un pokémon para expertos o ...,9.0,"[{'label': '2 stars', 'score': 0.3973818719387...",3.29,'4 starts',4.50
3104,meristation,https://as.com/meristation/2009/02/03/analisis...,Will van Dijk,Pikmin - New Play Control,Nintendo,Strategy,Wii,Poco a poco se van mitigando las dudas sobre l...,7.0,"[{'label': '3 stars', 'score': 0.5458763837814...",3.71,'4 starts',3.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6195,revogamers,https://www.revogamers.net/analisis-w/neon-chr...,Arturo Albero,Neon Chrome,None,None,Nintendo Switch,10tons ha traído a Nintendo Switch sus tres ju...,6.0,"[{'label': '4 stars', 'score': 0.4140677750110...",4.00,'3 starts',3.00
6196,Gamereactor,https://www.gamereactor.es/the-curious-tale-of...,Mikael Sundberg,The Curious Tale of the Stolen Pets,None,Puzle,None,"Fast Travel Games es un estudio pequeñito, per...",6.0,"[{'label': '4 stars', 'score': 0.4538985490798...",3.67,'3 starts',3.00
6197,Vandal,https://vandal.elespanol.com/analisis/pc/my-fr...,Juan Rubio,My Friend Pedro - Análisis,Devolver Digital,Acción Arcade,PC Switch Xbox One PS4,"Cada género tiene sus particularidades y, por ...",8.0,"[{'label': '4 stars', 'score': 0.6285300254821...",4.00,'4 starts',4.00
6198,meristation,https://as.com/meristation/2010/03/24/analisis...,Salva Fernàndez,Prison Break: The Conspiracy,Deep Silver,"Acción, Aventura",PC 360 PS3,Las adaptaciones van y vienen. De Libros a vid...,5.0,"[{'label': '4 stars', 'score': 0.4500963985919...",3.22,'2 starts',2.50
